# (5)사용자 정의 함수 및 apply, groupby
- 판다스와 넘파이의 집계 함수 외, 필요한 결과를 얻기 위해 집계함수를 직접 작성해야할 경우 apply 필요

In [1]:
import pandas as pd
import numpy as np
nls97 = pd.read_csv('data/nls97b.csv')
nls97.set_index('personid', inplace=True)

### 사분위범위(interquartile range)를 정의하는 함수 작성

In [8]:
def iqr(x):
    return x.quantile(0.75) - x.quantile(0.25)

In [9]:
# 함수 실행
aggdict = {'weeksworked16':['count', 'mean', iqr], 'childathome':['count', 'mean', iqr]}

nls97.groupby(['highestdegree']).agg(aggdict)

weeksworked16                  childathome               
                        count       mean   iqr       count      mean  iqr
highestdegree                                                            
0. None                   555  26.814414  53.0         439  1.840547  3.0
1. GED                    928  33.599138  53.0         693  1.701299  3.0
2. High School           2809  38.344251  34.0        1961  1.863845  2.0
3. Associates             627  40.614035  20.0         428  1.967290  2.0
4. Bachelors             1411  44.931254   1.0         827  1.896010  1.0
5. Masters                556  46.327338   0.0         333  1.918919  1.0
6. PhD                     53  45.943396   0.0          32  2.062500  2.0
7. Professional           103  45.912621   2.0          57  1.807018  1.0

### 선택한 요약통계를 시리즈로 반환하는 함수 정의

In [10]:
def gettots(x):
    out = {}
    out['qr1'] = x.quantile(0.25)
    out['med'] = x.median()
    out['qr3'] = x.quantile(0.75)
    out['count'] = x.count()
    return pd.Series(out)

In [11]:
# apply로 함수 실행
pd.options.display.float_format = '{:,.0f}'.format

nls97.groupby(['highestdegree'])['weeksworked16'].apply(gettots)

highestdegree         
0. None          qr1         0
                 med        27
                 qr3        53
                 count     555
1. GED           qr1         0
                 med        52
                 qr3        53
                 count     928
2. High School   qr1        19
                 med        53
                 qr3        53
                 count   2,809
3. Associates    qr1        33
                 med        53
                 qr3        53
                 count     627
4. Bachelors     qr1        52
                 med        53
                 qr3        53
                 count   1,411
5. Masters       qr1        53
                 med        53
                 qr3        53
                 count     556
6. PhD           qr1        53
                 med        53
                 qr3        53
                 count      53
7. Professional  qr1        51
                 med        53
                 qr3        53
                

### groupby 데이터프레임으로부터 생성한 인덱스 대신 디폴트 인덱스 사용을 위해, reset_index

In [12]:
nls97.groupby(['highestdegree'])['weeksworked06'].apply(gettots).reset_index()

,highestdegree,level_1,weeksworked06
0,0. None,qr1,5
1,0. None,med,34
2,0. None,qr3,52
3,0. None,count,703
4,1. GED,qr1,13
5,1. GED,med,42
6,1. GED,qr3,52
7,1. GED,count,"1,104"
8,2. High School,qr1,31
9,2. High School,med,52


### 요약변수에 기초한 열을 생성하기 위해 unstack과 엮기
- highestdegree 값을 인덱스로 하고 집계값이 열이 되는 데이터프레임 생성

In [15]:
nlssum = nls97.groupby(['highestdegree'])['weeksworked06'].apply(gettots).unstack()
nlssum

,qr1,med,qr3,count
highestdegree,,,,
0. None,5,34,52,703
1. GED,13,42,52,"1,104"
2. High School,31,52,52,"3,368"
3. Associates,34,52,52,722
4. Bachelors,38,52,52,"1,642"
5. Masters,39,52,52,601
6. PhD,29,50,52,53
7. Professional,4,29,49,117


In [16]:
nlssum.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0. None to 7. Professional
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   qr1     8 non-null      float64
 1   med     8 non-null      float64
 2   qr3     8 non-null      float64
 3   count   8 non-null      float64
dtypes: float64(4)
memory usage: 320.0+ bytes
